## Import packages

In [ ]:
import numpy as np
import numpy.random as npr
from matplotlib import pyplot as plt
%matplotlib inline
from matplotlib import cm
from scipy import io
import time

In [ ]:
# %load_ext autoreload
# %autoreload 2

## Load Data

In [ ]:
load_folder='/Users/jig289/Dropbox/SCA/SSA/data/'
data=io.loadmat(load_folder+'Balboa_outAndBack_redYellowConds_rawRates')

In [ ]:
data_array=data['data'] #neural data

In [ ]:
data_array.shape

## Preprocess data

In [ ]:
#Downsample data (using a factor of 10 here)
data_downsamp=data_array[:,:,np.arange(0,data_array.shape[2],10)]

#Concatenate all the conditions (so the matrix is size N x TC instead of C x N x T)
data_concat=data_downsamp.swapaxes(0,1).reshape([data_downsamp.shape[1],data_downsamp.shape[0]*data_downsamp.shape[2]])

#Maximum firing rates of each neuron (for normalizing later)
max_fr=np.max(data_concat,axis=1)[:,None]

**Data w/o subtracting mean**

In [ ]:
#Soft normalize (divide each neuron by its max FR + 5)
data_norm=data_concat/(max_fr+5)

**Data w/ subtracting neurons' means**

In [ ]:
data_snm_norm=data_norm-np.mean(data_norm,axis=1)[:,None]

**Data w subtracting condition mean**

In [ ]:
#Subtract condition mean
data_scm=data_downsamp-np.mean(data_downsamp,axis=0)[None,:,:]

#Concatenate all the conditions (so the matrix is size N x TC instead of C x N x T)
data_scm_concat=data_scm.swapaxes(0,1).reshape([data_scm.shape[1],data_scm.shape[0]*data_scm.shape[2]])

#Soft normalize (divide each neuron by its max FR + 5)
data_scm_norm=data_scm_concat/(max_fr+5)

## Fit models

#### Choose data and hyperparameters

In [ ]:
#Note that model requires (T x N) input rather than (N x T), which is why there are transposes below

fit_data=np.copy(data_scm_norm.T)     #subtract condition mean data
# fit_data=np.copy(data_norm.T)       #no mean subtracted data
# fit_data=np.copy(data_snm_norm.T)   #subtract neuron mean data

In [ ]:
fit_data.shape